In [6]:
import cv2
import mediapipe as mp

# Initialize MediaPipe hands and drawing utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Function to classify gestures
def classify_hand_gesture(hand_landmarks):
    # We will use the tip of the fingers (landmarks: 8, 12, 16, 20) and the thumb (landmark: 4)
    # For simplicity, let's define:
    # Rock: All fingers are closed (folded)
    # Paper: All fingers are open
    # Scissors: Index and middle finger are open, others are closed

    # Get landmarks
    thumb_tip = hand_landmarks.landmark[4].y
    index_tip = hand_landmarks.landmark[8].y
    middle_tip = hand_landmarks.landmark[12].y
    ring_tip = hand_landmarks.landmark[16].y
    pinky_tip = hand_landmarks.landmark[20].y

    # Get the base of each finger (second joint from tip)
    index_base = hand_landmarks.landmark[5].y
    middle_base = hand_landmarks.landmark[9].y
    ring_base = hand_landmarks.landmark[13].y
    pinky_base = hand_landmarks.landmark[17].y

    # Determine if fingers are open (tip is above the base) or closed
    thumb_open = hand_landmarks.landmark[4].x < hand_landmarks.landmark[6].x
    index_open = index_tip < index_base
    middle_open = middle_tip < middle_base
    ring_open = ring_tip < ring_base
    pinky_open = pinky_tip < pinky_base

    # Classify gesture
    if not thumb_open and not index_open and not middle_open and not ring_open and not pinky_open:
        return "Rock"
    elif index_open and middle_open and not ring_open and not pinky_open:
        return "Scissors"
    elif index_open and middle_open and ring_open and pinky_open:
        return "Paper"
    else:
        return "Unknown"

# Initialize webcam
cap = cv2.VideoCapture(1)

# Create MediaPipe hands detector
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a natural selfie-view display
        frame = cv2.flip(frame, 1)

        # Convert the frame to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hands
        result = hands.process(rgb_frame)

        # If hand landmarks are detected
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Classify the gesture
                gesture = classify_hand_gesture(hand_landmarks)

                # Display the classified gesture on the frame
                cv2.putText(frame, f'Gesture: {gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('Rock, Paper, Scissors Gesture Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
